# Автоматизация

<b>Техническое задание:<b/>
   
*Бизнес-задача*: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
    
*Насколько часто предполагается пользоваться дашбордом*: не реже, чем раз в неделю;
    
*Кто будет основным пользователем дашборда*: менеджеры по анализу контента;
    
*Состав данных для дашборда*:
- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;

*По каким параметрам данные должны группироваться*:
- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;

*Характер данных*:
- История событий по темам карточек — абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников — относительные величины (% событий)
- Соответствия тем источников темам карточек - абсолютные величины;

*Важность*: все графики имеют равную важность;

*Источники данных для дашборда*: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);

*База данных, в которой будут храниться агрегированные данные*: дополнительные агрегированные таблицы в БД zen;

*Частота обновления данных*: один раз в сутки, в полночь по UTC;


Импортируем необходимые библиотеки, зададим параметры для подключения к БД, сформируем строку подключения к ней

In [12]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

Сформируем sql-запрос

In [9]:
query = ''' SELECT *
            FROM log_raw
        '''

Выполним запрос и сохраним результат его выполнения в DataFrame

In [10]:
data_raw = pd.io.sql.read_sql(query, con = engine) 

In [11]:
data_raw.head()

,event_id,age_segment,event,item_id,item_topic,item_type,source_id,source_topic,source_type,ts,user_id
0,2931262,18-25,show,1793260,Красота,native,8388799,История,publisher,1569349657638,2185026
1,2931277,18-25,show,7465641,Культура,native,5837959,История,publisher,1569349657971,6845238
2,2931285,18-25,show,406073,История,url,2335872,Сад и дача,domain,1569349658408,5244363
3,2931247,26-30,show,752665,Путешествия,native,6963489,Здоровье,publisher,1569349657189,7553074
4,2931278,26-30,click,6843081,Юмор,url,5427748,Семейные отношения,domain,1569349658054,1317585


В результате выполнения запроса мы получили "сырые данные". Для дальнейшего анализа будет использоваться агрегирующая таблица с необходимой для построения дашборда информацией. Напишим код для выгрузки данных из этой таблицы. 

In [20]:
query_2 = ''' SELECT *
            FROM dash_visits
        '''

In [21]:
dash_visits = pd.io.sql.read_sql(query_2, con = engine) 

In [22]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


Выгрузим данные из таблицы dash_visits в файл dash_visits.csv

In [26]:
import os  
os.makedirs('D:/Downloads', exist_ok=True)  
dash_visits.to_csv('D:/Downloads/dash_visits.csv')  

[Дашборд] (https://public.tableau.com/app/profile/yuliya8789/viz/Automationproject_16561721493160/_)